In [105]:
# Import necessary libraries
from flask import Flask, request, jsonify
import joblib
import numpy as np
from werkzeug.exceptions import BadRequest
import logging
from threading import Thread
import socket
import pandas as pd
import requests
import json

In [106]:
ML_model= joblib.load('/Users/rohit.jishtu/Documents/GitHub/NewMachine/Projects/ML Projects/Project 7 - Bank Marketing Data/BankDtreeModel.joblib')

In [108]:
# Import necessary libraries
from flask import Flask, request, jsonify
import joblib
import numpy as np
from werkzeug.exceptions import BadRequest
import logging
from threading import Thread
import socket

# Configure logging
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Function to find an available port
def find_free_port():
    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(('', 0))
        s.listen(1)
        port = s.getsockname()[1]
    return port

# Initialize Flask application
app = Flask(__name__)

# Load the model when starting the server
model = None

def load_model():
    global model
    try:
        model = ML_model
        logger.info("Model loaded successfully!")
        return True
    except FileNotFoundError:
        logger.error("Error: model.joblib not found. Please ensure model file exists.")
        return False
    except Exception as e:
        logger.error(f"Error loading model: {str(e)}")
        return False

# Load model at startup
load_model()

@app.route('/predict', methods=['POST'])
def predict():
    if model is None:
        return jsonify({
            'status': 'error',
            'message': 'Model not loaded'
        }), 503
    
    try:
        # Get data from request
        data = request.get_json()
        
        # Validate input
        if not data or 'features' not in data:
            raise BadRequest('Request must include "features" key')
            
        features = np.array(data['features']).reshape(1, -1)
        
        # Make prediction
        prediction = model.predict(features)
        
        # Return prediction as JSON
        return jsonify({
            'status': 'success',
            'prediction': prediction.tolist()
        })
        
    except Exception as e:
        logger.error(f"Prediction error: {str(e)}")
        return jsonify({
            'status': 'error',
            'message': str(e)
        }), 400

@app.route('/health', methods=['GET'])
def health():
    return jsonify({
        'status': 'healthy', 
        'model_loaded': model is not None
    })

# Find an available port
PORT = find_free_port()

# Function to run the Flask app in a thread
def run_app():
    app.run(host='127.0.0.1', port=PORT, debug=False, use_reloader=False)

# Start the Flask app in a separate thread
server_thread = Thread(target=run_app)
server_thread.daemon = True
server_thread.start()
print(f"Flask server is running on http://127.0.0.1:{PORT}")

INFO:__main__:Model loaded successfully!


Flask server is running on http://127.0.0.1:49597
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:49597
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 16:23:47] "GET /health HTTP/1.1" 200 -
/Users/rohit.jishtu/Library/Python/3.9/lib/python/site-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
ERROR:__main__:Prediction error: X has 4 features, but DecisionTreeClassifier is expecting 41 features as input.
INFO:werkzeug:127.0.0.1 - - [25/Dec/2024 16:23:47] "POST /predict HTTP/1.1" 400 -


In [111]:
import requests

# Get the port number from the previous output
PORT = 49597  # Replace this with the port number shown in the output

# Test the health endpoint
response = requests.get(f'http://127.0.0.1:{PORT}/health')
print("Health check:", response.json())



Health check: {'model_loaded': True, 'status': 'healthy'}
Prediction: {'message': 'X has 4 features, but DecisionTreeClassifier is expecting 41 features as input.', 'status': 'error'}


In [ ]:
# Test the predict endpoint
data = {"features": [1, 2, 3, 4]}
response = requests.post(f'http://127.0.0.1:{PORT}/predict', json=data)
print("Prediction:", response.json())